In [1]:
import os
import pandas as pd
import shutil
import tempfile
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab

import numpy as np
import PIL
from PIL import Image, ImageChops, ImageDraw
import torch
from torch import nn
import torchvision.models
import cv2
from skimage.morphology import skeletonize

from pathlib import Path

import tensorflow as tf
from tensorflow import keras
import matplotlib.patches as patches

from tqdm import tqdm

In [2]:
!nvidia-smi

Fri Jun 10 19:08:23 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.129.06   Driver Version: 470.129.06   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:06:00.0 Off |                    0 |
| N/A   69C    P0   134W / 149W |  10904MiB / 11441MiB |     82%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla K80           Off  | 00000000:07:00.0 Off |                    0 |
| N/A   

In [3]:
segmentation_classifier = keras.models.load_model('models/MIMIC-256x25680-20-split-resnet-Float16_2-race_detection_rop_seg_data_rop_seg-0.001_20220321-054140_epoch:011.hdf5')
segmentation_classifier.summary()

The dtype policy mixed_float16 may run slowly because this machine does not have a GPU. Only Nvidia GPUs with compute capability of at least 7.0 run quickly with mixed_float16.
If you will use compatible GPU(s) not attached to this host, e.g. by running a multi-worker model, you can ignore this warning. This message will only be logged once
Instructions for updating:
Use tf.keras.mixed_precision.LossScaleOptimizer instead. LossScaleOptimizer now has all the functionality of DynamicLossScale
Model: "functional_11"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
bn_data (BatchNormalization)    (None, 256, 256, 3)  9           input_3[0][0]       

### Preprocessing

In [ ]:
def prepare_for_classification(img_id, img_path, race, radius):
    
    if (radius != None):
        img = Image.open(img_path + str(race) + '/' + str(radius) + '/' + str(img_id) + '.bmp')
    else: # case of original images
        img = Image.open(img_path + str(img_id) + '.bmp')
        
    arr = np.array(img)
    channels = np.repeat(resized[:, :, np.newaxis], 3, axis=2).reshape((1,256,256,3))
    
    return channels

# wi

In [ ]:
def combine_images(data, img_path, race, radius):
    
    img_arr = np.zeros([1, 256, 256, 3], dtype='int')

    for i in tqdm(range(len(data))):
        img_id = data['image_id'][i] # because of this, predictions are in the right order, IMPORTANT
        # original preds
        img_prepared = prepare_for_classification(img_id, img_path, race, radius)
        img_arr = np.concatenate((img_arr, img_prepared), axis = 0)
        
    img_arr = img_arr[1:] # removing the unnecessary first element
    
    return img_arr